In [2]:
import math
from random import randrange,choice
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Experimento

In [ ]:
ruta = '/content/drive/MyDrive/Doctorado/Articulos/04. Sistema de identificación/Datos/'

from sklearn.model_selection import  train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import IsolationForest

df_all = pd.read_excel(ruta+'df_all.xlsx')
df_all_pesaje = pd.read_excel('/content/drive/MyDrive/Doctorado/Articulos/04. Sistema de identificación/Datos_pesaje/Datos_pesaje.xlsx')
df_all = df_all.reset_index()
df_all = df_all.drop(['Unnamed: 0','index'],axis=1)
df_all_pesaje = df_all_pesaje.drop(['Unnamed: 0'],axis=1)

X = df_all.loc[:,df_all.columns!='peso'].values
y = df_all.loc[:,'peso'].values

df_all_pesaj=df_all_pesaje

df_all_pesaje = pd.get_dummies(df_all_pesaj)
pesaje = []

for i in range(len(df_all_pesaje)):
  pesaje.append([
                 df_all_pesaje['dia'][i],
                 df_all_pesaje['peso'][i],
                 df_all_pesaje['raza_AC'][i],
                 df_all_pesaje['raza_BAC'][i],
                 df_all_pesaje['raza_BC'][i],
                 df_all_pesaje['raza_CAC'][i],
                 df_all_pesaje['raza_CC'][i],
                 df_all_pesaje['genero_Hembra'][i],
                 df_all_pesaje['genero_Macho'][i]])

for i in range(len(pesaje)):
  predicciones = []

  for j in range(30):
    seed = randrange(1,9999999)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
    modelo = RandomForestRegressor(n_estimators=10,max_depth=100, random_state = seed)
    modelo.fit(X_train, y_train)
    predictions = modelo.predict(X_test)
    inputs = {
      'dia': [pesaje[j][0]],
      'raza_AC': [pesaje[j][2]],
      'raza_BAC': [pesaje[j][3]],
      'raza_BC': [pesaje[j][4]],
      'raza_CAC': [pesaje[j][5]],
      'raza_CC': [pesaje[j][6]],
      'genero_Hembra': [pesaje[j][7]],
      'genero_Macho': [pesaje[j][8]],
    }
    inputs = pd.DataFrame(inputs)
    predicciones.append(modelo.predict(inputs)[0])
  pesaje[i].append([predicciones,min(predicciones),max(predicciones)])
Experimento = pd.DataFrame(pesaje,columns=['dia','peso_real','raza_AC','raza_BAC','raza_BC','raza_CAC','raza_CC','genero_Hembra','genero_Macho','predicciones'])
Experimento['min_predicion'] = 0
Experimento['max_predicion'] = 0

for i in range(len(Experimento)):
  Experimento['min_predicion'][i] = Experimento['predicciones'][i][1]
  Experimento['max_predicion'][i] = Experimento['predicciones'][i][2]
  Experimento['predicciones'][i] = Experimento['predicciones'][i][0]

Experimento['anomalia']=""
Experimento['masinfo_anomalia'] = ""
for i in range(len(Experimento)):
  out_li = []
  for j in Experimento['predicciones'][i]:
    out_li.append([round(j)])
  out_li.append([Experimento['peso_real'][i]])
  lof = IsolationForest(contamination=0.01)
  yhat = lof.fit_predict(out_li)
  if yhat[-1]==1:
    Experimento['anomalia'][i] = "normal"
    Experimento['masinfo_anomalia'][i] = "-"
  else:
    Experimento['anomalia'][i] = "abnormal"
    if Experimento['peso_real'][i]>=Experimento['min_predicion'][i] and Experimento['peso_real'][i]<=Experimento['max_predicion'][i]:
      Experimento['masinfo_anomalia'][i] = "-"
    if Experimento['peso_real'][i]<Experimento['min_predicion'][i]:
      Experimento['masinfo_anomalia'][i] = "down"
    if Experimento['peso_real'][i]>Experimento['max_predicion'][i]:
      Experimento['masinfo_anomalia'][i] = "up"


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without featu

In [ ]:
Experimento.to_excel(ruta+'Experimento.xlsx')

In [ ]:
Experimento

###Dia 1200, Peso ganado 364, Intervalo de 395 a 408

In [ ]:
dia = 1200
peso_real = 364
predicciones = []
for i in range(30):
  seed = randrange(1,9999999)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
  modelo = RandomForestRegressor(n_estimators=10,max_depth=100, random_state = seed)
  modelo.fit(X_train, y_train)
  predictions = modelo.predict(X_test)
  inputs = {
    'dia': [dia],
    'raza_AC': [0],
    'raza_BAC': [1],
    'raza_BC': [0],
    'raza_CAC': [0],
    'raza_CC': [0],
    'genero_Hembra': [1],
    'genero_Macho': [0],
  }

  inputs = pd.DataFrame(inputs)
  predicciones.append(modelo.predict(inputs))

from sklearn.ensemble import IsolationForest
out_li = []
out_li2 = []
real = [peso_real]
for i in range(len(predicciones)):
  out_li.append([round(predicciones[i][0])])
  out_li2.append(round(predicciones[i][0]))
print("Peso Min: "+str(min(out_li2)))
print("Peso Max: "+str(max(out_li2)))
print("Peso Real: "+str(peso_real))
out_li.append(real)
lof = IsolationForest(contamination=0.01)
yhat = lof.fit_predict(out_li)
print(yhat)

Peso Min: 391
Peso Max: 408
Peso Real: 364
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1]


###Dia 822, Peso ganado 326, Intervalo de 350 a 365

In [ ]:
dia = 822
peso_real = 326
predicciones = []
for i in range(30):
  seed = randrange(1,9999999)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
  modelo = RandomForestRegressor(n_estimators=10,max_depth=100, random_state = seed)
  modelo.fit(X_train, y_train)
  predictions = modelo.predict(X_test)
  inputs = {
    'dia': [dia],
    'raza_AC': [0],
    'raza_BAC': [1],
    'raza_BC': [0],
    'raza_CAC': [0],
    'raza_CC': [0],
    'genero_Hembra': [1],
    'genero_Macho': [0],
  }

  inputs = pd.DataFrame(inputs)
  predicciones.append(modelo.predict(inputs))

from sklearn.ensemble import IsolationForest
out_li = []
out_li2 = []
real = [peso_real]
for i in range(len(predicciones)):
  out_li.append([round(predicciones[i][0])])
  out_li2.append(round(predicciones[i][0]))
print("Peso Min: "+str(min(out_li2)))
print("Peso Max: "+str(max(out_li2)))
print("Peso Real: "+str(peso_real))
out_li.append(real)
lof = IsolationForest(contamination=0.01)
yhat = lof.fit_predict(out_li)
print(yhat)

Peso Min: 350
Peso Max: 365
Peso Real: 326
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1 -1]


###Dia 650, Peso ganado 322, Intervalo de 395 a 408

In [ ]:
dia = 650
peso_real = 322
predicciones = []
for i in range(30):
  seed = randrange(1,9999999)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = seed)
  modelo = RandomForestRegressor(n_estimators=10,max_depth=100, random_state = seed)
  modelo.fit(X_train, y_train)
  predictions = modelo.predict(X_test)
  inputs = {
    'dia': [dia],
    'raza_AC': [0],
    'raza_BAC': [1],
    'raza_BC': [0],
    'raza_CAC': [0],
    'raza_CC': [0],
    'genero_Hembra': [1],
    'genero_Macho': [0],
  }

  inputs = pd.DataFrame(inputs)
  predicciones.append(modelo.predict(inputs))

from sklearn.ensemble import IsolationForest
out_li = []
out_li2 = []
real = [peso_real]
for i in range(len(predicciones)):
  out_li.append([round(predicciones[i][0])])
  out_li2.append(round(predicciones[i][0]))
print("Peso Min: "+str(min(out_li2)))
print("Peso Max: "+str(max(out_li2)))
print("Peso Real: "+str(peso_real))
out_li.append(real)
lof = IsolationForest(contamination=0.01)
yhat = lof.fit_predict(out_li)
print(yhat)

Peso Min: 314
Peso Max: 327
Peso Real: 322
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1]
